In [57]:
import pandas as pd

# read in the file
df = pd.read_csv('2020 week 1/PD 2020 WK 1 Input - Sheet1.csv')

# replace nulls with zeroes
df['Profit'].fillna(0, inplace=True)

In [58]:
# extract the hierarchy from the Item and copy it
df['Hierarchy'] = df['Item'].str.extract('([\d\.]+?)\.? .*')
df['Hierarchy2'] = df['Hierarchy']
df['Level'] = df['Hierarchy'].str.count('\.')

maxLevel = df['Level'].max()
print(maxLevel)

2


In [59]:
df

,Item,Profit,Hierarchy,Hierarchy2,Level
0,1. Bar Soap,0.0,1,1,0
1,1.1 Opaque,0.0,1.1,1.1,1
2,1.1.1 Lavender,10778.0,1.1.1,1.1.1,2
3,1.1.2 Honey,-6103.0,1.1.2,1.1.2,2
4,1.1.3 Orange,5422.0,1.1.3,1.1.3,2
5,1.1.4 Mint,6301.0,1.1.4,1.1.4,2
6,1.2 Transparent,0.0,1.2,1.2,1
7,1.2.1 Lavender,10769.0,1.2.1,1.2.1,2
8,1.2.2 Honey,-270.0,1.2.2,1.2.2,2
9,1.2.3 Orange,-4981.0,1.2.3,1.2.3,2


In [60]:
# iterate through the hierarchy levels, creating subtotals at each level
dfSubtotal = None
for i in range(maxLevel, 0, -1):
    # remove a layer of hierarchy by stripping a number + a period(if there) 
    #in each oteration of the loop.
    #
    df['Hierarchy2'] = df['Hierarchy2'].str.extract('(.*?)\.\d+$')

    #
    dfSubtotal = pd.concat([dfSubtotal,
                            df[df['Level'] == maxLevel].groupby(df['Hierarchy2'], as_index=True)['Profit'].sum()])
    print(df[df['Level'] == maxLevel][['Item','Hierarchy2', 'Level', 'Profit']])
    print(dfSubtotal)

              Item Hierarchy2  Level   Profit
2   1.1.1 Lavender        1.1      2  10778.0
3      1.1.2 Honey        1.1      2  -6103.0
4     1.1.3 Orange        1.1      2   5422.0
5       1.1.4 Mint        1.1      2   6301.0
7   1.2.1 Lavender        1.2      2  10769.0
8      1.2.2 Honey        1.2      2   -270.0
9     1.2.3 Orange        1.2      2  -4981.0
10      1.2.4 Mint        1.2      2   -172.0
13  2.1.1 Lavender        2.1      2  -1740.0
14     2.1.2 Honey        2.1      2  14520.0
15    2.1.3 Orange        2.1      2   1551.0
16      2.1.4 Mint        2.1      2  14479.0
18  2.2.1 Lavender        2.2      2  -4302.0
19     2.2.2 Honey        2.2      2  -1566.0
20    2.2.3 Orange        2.2      2   1187.0
21      2.2.4 Mint        2.2      2   3090.0
24     3.1.1 Green        3.1      2  -9507.0
25    3.1.2 Yellow        3.1      2  -2863.0
26       3.1.3 Red        3.1      2  -3918.0
28     3.2.1 Stone        3.2      2  -7070.0
29     3.2.2 White        3.2     

In [52]:
# join subtotals back to the main dataframe and update the Profit
df = pd.merge(df, dfSubtotal, how='left', left_on='Hierarchy', right_index=True)
df['Profit'] = df['Profit_x'] + df['Profit_y'].fillna(0)
df

,Item,Profit_x,Hierarchy,Hierarchy2,Level,Profit_y,Profit
0,1. Bar Soap,0.0,1,NaN,0,21744.0,21744.0
1,1.1 Opaque,0.0,1.1,NaN,1,16398.0,16398.0
2,1.1.1 Lavender,10778.0,1.1.1,1,2,NaN,10778.0
3,1.1.2 Honey,-6103.0,1.1.2,1,2,NaN,-6103.0
4,1.1.3 Orange,5422.0,1.1.3,1,2,NaN,5422.0
5,1.1.4 Mint,6301.0,1.1.4,1,2,NaN,6301.0
6,1.2 Transparent,0.0,1.2,NaN,1,5346.0,5346.0
7,1.2.1 Lavender,10769.0,1.2.1,1,2,NaN,10769.0
8,1.2.2 Honey,-270.0,1.2.2,1,2,NaN,-270.0
9,1.2.3 Orange,-4981.0,1.2.3,1,2,NaN,-4981.0


In [53]:
# add the spacing to the Item
df['Item'] = [' ' * 5 * x for x in df['Level']] + df['Item']
df['Item']

0                    1. Bar Soap
1                     1.1 Opaque
2                 1.1.1 Lavender
3                    1.1.2 Honey
4                   1.1.3 Orange
5                     1.1.4 Mint
6                1.2 Transparent
7                 1.2.1 Lavender
8                    1.2.2 Honey
9                   1.2.3 Orange
10                    1.2.4 Mint
11                2. Liquid Soap
12                    2.1 Opaque
13                2.1.1 Lavender
14                   2.1.2 Honey
15                  2.1.3 Orange
16                    2.1.4 Mint
17               2.2 Transparent
18                2.2.1 Lavender
19                   2.2.2 Honey
20                  2.2.3 Orange
21                    2.2.4 Mint
22                3. Accessories
23         3.1 Exfoliating Glove
24                   3.1.1 Green
25                  3.1.2 Yellow
26                     3.1.3 Red
27               3.2 Soap Holder
28                   3.2.1 Stone
29                   3.2.2 White
30        

In [55]:
# clean up columns
df = df[['Item', 'Profit']]
df

,Item,Profit
0,1. Bar Soap,21744.0
1,1.1 Opaque,16398.0
2,1.1.1 Lavender,10778.0
3,1.1.2 Honey,-6103.0
4,1.1.3 Orange,5422.0
5,1.1.4 Mint,6301.0
6,1.2 Transparent,5346.0
7,1.2.1 Lavender,10769.0
8,1.2.2 Honey,-270.0
9,1.2.3 Orange,-4981.0
